# Centralised Learning

Set the module directory to import python files (RUN JUST ONCE)

In [1]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



['/home/victor/_bcfl/fabric-federated-learning/federated-learning/client/notebooks', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python311.zip', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/lib-dynload', '', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages', '/home/victor/_bcfl/fabric-federated-learning/federated-learning']


In [2]:
import torch
print(f"Cuda available: {torch.cuda.is_available()}")
# Get the name of the CUDA device
print(torch.cuda.get_device_name(0))

try:
    print(f"major and minor cuda capability of the device: {torch.cuda.get_device_capability()}")
except Exception:
    print("No Cuda available")
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Using device: %s'%device)# Check if CUDA is available and set the default tensor type to CUDA
if torch.cuda.is_available():
    torch.set_default_device('cuda')
    print("Cuda set as default device")
else:
    torch.set_default_device('cpu')
    print("Cuda not available, CPU set as default device")

Cuda available: True
NVIDIA GeForce MX150
major and minor cuda capability of the device: (6, 1)
Using device: cuda
Cuda set as default device


Training a sample Perceptron to try the blockchain integration

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from client.model.perceptron import Perceptron
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

# torch.set_default_device('cpu')


n_features = 10  # Example number of input features
num_classes = 1  # Example number of classes
# model = Perceptron(n_features, num_classes)  # Instantiate the model (on the default device
model = Perceptron(n_features)  # Instantiate the model (on the default device

loss_function = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent
# Example (dummy) training data
dummy_inputs = torch.randn(100, n_features)  # 100 samples, n_features each
print(dummy_inputs.device)
dummy_targets = torch.randint(0, 2, (100, 1)).float()  # Binary target values (0 or 1)

# Training loop
for epoch in range(5):  # Number of epochs
    optimizer.zero_grad()  # Clearing the gradients
    outputs = model(dummy_inputs)  # Forward pass
    loss = loss_function(outputs, dummy_targets)  # Compute loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights

    if epoch % 1 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')



cuda:0
Epoch 0, Loss: 0.696426510810852
Epoch 1, Loss: 0.6962855458259583
Epoch 2, Loss: 0.6961452960968018
Epoch 3, Loss: 0.6960055232048035
Epoch 4, Loss: 0.6958665251731873


Loading CIFAR DATASET

In [11]:
from client.dataloader import get_cifar10_dataloaders, get_cifar10_datasets

root = 'client/data/'
num_training= 49000
num_validation =1000
batch_size = 200
train_dataset, val_dataset, test_dataset = get_cifar10_datasets(root, num_training, num_validation)
train_loader, val_loader, test_loader = get_cifar10_dataloaders(root, batch_size, num_training, num_validation, device)


Files already downloaded and verified
Files already downloaded and verified


Create model and train

In [14]:
from client.model.perceptron import MultiLayerPerceptron
from client.train import train
from client.utils import weights_init, update_lr


input_size = 32 * 32 * 3
hidden_size = [50]
num_classes = 10
num_epochs = 1
learning_rate = 1e-3
learning_rate_decay = 0.95
reg=0.001

train_flag = True

model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model.to(device)
print(model)

dataloaders = {
    'train': train_loader,
    'validation': val_loader,
    'test': test_loader
}
    
# Training
model.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=reg)

train(model, dataloaders, 'client/models/', criterion, optimizer, learning_rate, learning_rate_decay, input_size, num_epochs, device)

MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
Epoch [1/3], Step [1/245], Loss: 2.3026
Epoch [1/3], Step [2/245], Loss: 2.3007
Epoch [1/3], Step [3/245], Loss: 2.2931
Epoch [1/3], Step [4/245], Loss: 2.2820
Epoch [1/3], Step [5/245], Loss: 2.2712
Epoch [1/3], Step [6/245], Loss: 2.2492
Epoch [1/3], Step [7/245], Loss: 2.2236
Epoch [1/3], Step [8/245], Loss: 2.2153
Epoch [1/3], Step [9/245], Loss: 2.2060
Epoch [1/3], Step [10/245], Loss: 2.1720
Epoch [1/3], Step [11/245], Loss: 2.1496
Epoch [1/3], Step [12/245], Loss: 2.1400
Epoch [1/3], Step [13/245], Loss: 2.1210
Epoch [1/3], Step [14/245], Loss: 2.0502
Epoch [1/3], Step [15/245], Loss: 2.0776
Epoch [1/3], Step [16/245], Loss: 2.0475
Epoch [1/3], Step [17/245], Loss: 2.0183
Epoch [1/3], Step [18/245], Loss: 2.0669
Epoch [1/3], Step [19/245], Loss: 2.0755
Epoch [1/3], Step [20/245], Loss: 2.0481

KeyboardInterrupt: 

Test the trained model

In [13]:
# Run the test code once you have your by setting train flag to false
# and loading the best model
best_model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
best_model = torch.load(pjoin(modelpath, 'model.ckpt'))
model.load_state_dict(best_model)
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################
       
        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * c

SyntaxError: incomplete input (2725263433.py, line 26)

Sample neural network code that checks if the tensors and model are running on GPU

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from client.model.perceptron import Perceptron
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

n_features = 10  # Example number of input features
model = Perceptron(n_features).to('cuda')  # Instantiate the model (on the default device
loss_function = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent
# Example (dummy) training data
dummy_inputs = torch.randn(100, n_features)  # 100 samples, n_features each
print(dummy_inputs.device)
dummy_targets = torch.randint(0, 2, (100, 1)).float()  # Binary target values (0 or 1)

# Training loop
for epoch in range(5):  # Number of epochs
    optimizer.zero_grad()  # Clearing the gradients
    start_event.record()
    outputs = model(dummy_inputs)  # Forward pass
    end_event.record()
    torch.cuda.synchronize()  # Wait for the events to be recorded!
    elapsed_time_ms = start_event.elapsed_time(end_event)
    print(f"Elapsed time (in milliseconds): {elapsed_time_ms}")
    loss = loss_function(outputs, dummy_targets)  # Compute loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights

    if epoch % 1 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')



cuda:0
Elapsed time (in milliseconds): 0.2836480140686035
Epoch 0, Loss: 0.7509689331054688
Elapsed time (in milliseconds): 0.23996800184249878
Epoch 1, Loss: 0.7505505681037903
Elapsed time (in milliseconds): 0.1724800020456314
Epoch 2, Loss: 0.7501339316368103
Elapsed time (in milliseconds): 0.151296004652977
Epoch 3, Loss: 0.7497190833091736
Elapsed time (in milliseconds): 0.11884800344705582
Epoch 4, Loss: 0.7493058443069458
